##                            Text analyzer


In [28]:
import ipywidgets as wg
from IPython.display import display

myName = wg.Text(value='write disease here...', description='Disease:   ')
N_abstract = wg.IntSlider(description='No. abstracts:', max=10000, value= 1000)
display(myName,N_abstract)

Text(value='write disease here...', description='Disease:   ')

IntSlider(value=1000, description='No. abstracts:', max=10000)

In [18]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/piotr/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/piotr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/piotr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from pymed import PubMed
import pandas as pd
import sklearn
import pickle
import os

import ipywidgets as wg
from IPython.display import display

In [20]:
print('Selected disease: ' + myName.value) 
print('Number of abstracts:  ' + str(N_abstract.value))

Selected disease: liver cancer
Number of abstracts:  1000


In [21]:

    text = myName.value
    max1 =int(N_abstract.value)  # ilosc zapytan
    #text = [text]

    #wporwadzenie zapytania do pubmed
    pubmed = PubMed(tool="MyTool", email="p.karabowicz@gmail.com")
    results1 = pubmed.query(text, max_results=max1)

    #przeksztalcenie wynikow zapytania na data frame
    lista_abstract_3=[]
    for i in results1:
        lista_abstract_3.append(i.abstract)
    import pandas as pd
    df_abstract = pd.DataFrame(lista_abstract_3, columns = ['abstracts'])
    df_abstract['abstracts_lower'] = df_abstract['abstracts'].str.lower()
    df_abstract_1 = df_abstract.dropna() #datafraame wynikow do analizy

    ###predykcja
    #from keras.models import load_model
    #BASE_DIR = os.path.dirname(os.path.abspath('finalized_model1.sav'))
    #BASE_DIR1 = '/home/piotr/drugfinal/static'
    #finalized_model= load_model(os.path.join(BASE_DIR, 'static/finalized_model1.sav'))
    
    #finalized_model= os.path.join(BASE_DIR, 'static/finalized_model1.sav')
    import pickle
    rnd = pickle.load(open('./finalized_model.sav', 'rb'))
    
    from sklearn.feature_extraction.text import CountVectorizer
    count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100)
    result1=count_vect.fit_transform(df_abstract_1['abstracts_lower'])
    result2 = rnd.predict(result1)
    df_abstract_1['class'] = result2 # dataframe z wynikami
    len_df = len(result2)

#unsupervised learning
    from gensim.models import Word2Vec
    from nltk.corpus import stopwords
    #przygotowanie tekstu do osadzania slow
    stop = stopwords.words('english')
    
    from nltk.tokenize import word_tokenize
    df_abstract_1['abstracts_stop'] = df_abstract_1['abstracts_lower'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df_abstract_1['tokenized'] = df_abstract_1.apply(lambda row: nltk.word_tokenize(row['abstracts_lower']), axis=1)
    model_ted1 = Word2Vec(sentences=df_abstract_1['tokenized'], size=200, window=10, min_count=1, workers=4, sg=0)

    #ekstrackja slow najbardziej podobnych do protein i target
    keys = ['protein', 'target']
    most_sim = model_ted1.wv.most_similar(positive = keys, topn=1000)
    #utworzenie tablicy ze slownika
    most_sim_key = []
    for w, n in most_sim:
        most_sim_key.append(w)
    #tagowanie tekstu i filtrowanie wedlug tagow
    post_tag_list = nltk.pos_tag(most_sim_key)

    listNN = []
    for w , k in post_tag_list:
        if k == "NN":
            listNN.append(w)

    listJJ = []
    for w , k in post_tag_list:
        if k == "JJ":
            listJJ.append(w)

    lista = [listNN, listJJ]

    flat_list = []
    for sublist in lista:
        for item in sublist:
            flat_list.append(item)
    #filtrowanie wzgledem bazy bialek
    #BASE_DIR2 = os.path.dirname(os.path.abspath('lista_bialek_bez1.txt'))
    file = open('./lista_bialek_bez1.txt', "r")
    wprowadzony_tekst = file.read()

    wt = wprowadzony_tekst.split(',')

    wt2 =[]
    for w in wt:
        w = w.replace("[", "")
        w = w.replace("]", "")
        w= w.replace("'", "")
        wt2.append(w)

    wt2 = [x.lower() for x in wt2 ]
    tablica_in =[] #lista wyekstrahowanych bialek
    for w in flat_list:
        if w in wt2:
            tablica_in.append(w)

    # predykcja dla zapytan z tablica_in
    import sklearn
    from sklearn.feature_extraction.text import CountVectorizer
    import time

    def query(list_target):
        pubmed = PubMed(tool="MyTool", email="p.karabowicz@gmail.com")
        lista=[]
        for w in list_target:
            time.sleep(1)
            lista.append(pubmed.query(w, max_results=20))
        return lista

    def lista_bastract_pred1(lista):
        lista_abstract_pred=[]
        for n in lista:
                lista_abstract_pred.append(n.abstract)
        return lista_abstract_pred

    def percent_true(ynew2):
        percent_true = round((list(ynew2).count(1))/len(list(ynew2))*100,1)
        return percent_true

    from sklearn.ensemble import RandomForestClassifier
    from sklearn.datasets import make_classification

    import pickle
    #rnd = pickle.load(open('C:\\Users\\UMB\\Desktop\\drugforest\\drugforest\\static\\finalized_model_32.sav', 'rb'))

    a = query(tablica_in)
    d = []
    for w in a:
        b = lista_bastract_pred1(w) ## ziterować
        d.append(b)

    d2=[]
    for g in d:
        d1 = list(filter(None.__ne__, g))
        d2.append(d1)

    count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100)
    lista_y=[]
    for w in d2:
        result1=count_vect.fit_transform(w)
        result2 = rnd.predict(result1)
        lista_y.append(result2)

    yy = []
    for y in lista_y:
        yy.append(percent_true(y))

    import pandas as pd
    data_tuples = list(zip(tablica_in,yy))
    df_list = pd.DataFrame(data_tuples, columns=['Protein','Score'])

    df_list = df_list.sort_values('Score', ascending = False)
    #df_list = df_list.reset_index()

    df_abstract_1 = df_abstract_1[['abstracts', 'class']]


/home/piotr/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/piotr/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
<ipython-input-21-52140cebb358>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_abstract_1['class'] = result2 # dataframe z wynikami
<ipython-input-21-52140cebb358>:42: SettingWithCopyWarning: 
A value is tr

In [22]:
df_list[['Protein', 'Score']]


,Protein,Score
8,pp2a,78.9
5,vdr,75.0
14,emt,75.0
4,mcs,70.0
7,hep,70.0
11,next,68.4
6,mec,65.0
9,ccr2,65.0
15,lkb1,63.2
10,e-cadherin,63.2
